<a href="https://colab.research.google.com/github/MarinaOhm/master_thesis/blob/main/RICE_FORECASTING_SCRIPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master thesis
## *Forecasting of Sales data on SKU level*
*Developed by Max Hedeman Gueniau, Niklas Madsen, and Marina Ohm*

# Install and import libraries

## Install libraries

In [ ]:
!pip install pystan numpy pandas matplotlib Cython setuptools
!pip install pmdarima statsmodels h5py keras-tuner
!pip install pystan~=2.14
!pip install --no-cache-dir fbprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py) ... done
  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running setup.py clean for fbprophet
Failed to build fbprophet
ERROR: Could not build wheels for fbprophet, which is required to install pyproject.toml-based projects


## Import libraries

In [ ]:
import pandas as pd, numpy as np, seaborn as sns, time, matplotlib.pyplot as plt, plotly.express as px, random, plotly.graph_objs as go, xgboost as xgb, lightgbm as lgb, tensorflow as tf
import math, warnings, pmdarima, statsmodels, pickle, time, logging, os, itertools, keras, keras_tuner as kt
from fbprophet import Prophet
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBRegressor
import prophet.diagnostics
from prophet.diagnostics import cross_validation, performance_metrics
from pmdarima.arima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
from datetime import datetime
from statsmodels.tsa.stattools import adfuller, acf
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import Objective
from pickle import dump,load
from sklearn.base import BaseEstimator, clone
from scipy.stats import uniform
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import Objective
from scipy.stats import norm
from matplotlib import rcParams
from google.colab import drive

warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('fbprophet').setLevel(logging.ERROR)
drive.mount('/content/drive')

Mounted at /content/drive


# Sales forecasting

## Support functions applied in the train/test function and the forecast function

In [ ]:
def import_data(path):
  """
  Import data through Gdrive path

  :parameters:
  - Path to data in Gdrive

  :return:
  - Imported data in DataFrame
  """

  path_data = path
  df = pd.read_csv(path_data)

  # Subset only necessary columns 
  df = df[['_ItemNumber','date_','Q']]

  return df


def lstm_prepros_seq(data, seq=15):
  """
  Function for preparing data for LSTM. Specifically by scaling it and converting it into sequences 

  :parameters:
  - Training and testing data

  :return:
  - Array x with sequential input data
  - Array y with sequential target data
  - Scalar object for invert scaling
  """

  # Scaling data 
  scaler = MinMaxScaler()
  data = scaler.fit_transform(data)

  # Defining objects to store input sequences and corresponding target values
  x = []
  y = []

  # Looping through the data. The -seq-1 part is to ensure there is enough data points to create an input sequence of length seq AND a corresponding target value
  for i in range(len(data)-seq-1):

    # Breaking the data into smaller 'windows' over overlapping sequences
    window = data[i:(i+seq),0] 
    x.append(window)
    y.append(data[i+seq,0]) 
  
  return np.array(x), np.array(y), scaler


def model_tuning(hp, train_lstm_x, train_lstm_y):
    """
    Function for hyperparameter tuning LSTM and finding the best performing NN architecture

    :parameters:
    - HP settings
    - Training data x values
    - Training data y values

    :return:
    - Best performing LSTM model
    """

    # Initiate sequential class
    model = Sequential()
    
    # Adding an LSTM layer to the model and testing different # of units ranging between a min and a max defined by a step. Input shape is defined by the shape of the train data
    model.add(LSTM(hp.Int('input_unit', min_value=32, max_value=512, step=32), return_sequences=True, input_shape=(train_lstm_x.shape[1],train_lstm_x.shape[2])))

    # Adding LSTM layers to the model and testing different # of units in each. 
    for i in range(hp.Int('n_layers', 1, 5)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units', min_value=32, max_value=512, step=32),return_sequences=True))
    
    # Adding final LSTM layer without return_sequences = True to ensure that a single value is returned for the next layer
    model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32)))

    # A dropout layer and testing different dropout values
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))

    # Adding a dense layer with 30 neurons added and with the activation function being either relu or sigmoid
    model.add(Dense(30, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))

    # Adding output layer with the activation function being either relu or sigmoid. The output layer returns one neuron, as the model is expected to output a single value.
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
   
    # Compiling model using ADAM and MSE
    model.compile(loss='mean_squared_error', optimizer='adam' ,metrics = ['mse'])
    
    return model
    
    

def preprocess(df, sku):
    """
    Preprocessing function to make the SKU level data ready for further modelling

    :parameters:
    - Full DataFrame
    - String with the itemnumber of the SKU to be processed

    :returns:
    - A preprocessed, and stationary SKU DataFrame
    - An object containing the level of differencing conducted

    """

    np.random.seed(42)

    # Subsetting SKU data from full df 
    sku_df = df[df['_ItemNumber']==sku].drop('_ItemNumber', axis=1)

    # Convert date_ column to datetime object and use as index
    sku_df['date_'] = pd.to_datetime(sku_df['date_'])
    sku_df.set_index('date_', inplace=True)

    # Resample df to weekly frequency and sum the values of the Q column
    sku_df = sku_df.resample('W', label='right', closed='right').sum()

    # Impute missing values using linear interpolation
    sku_df = sku_df.interpolate(method='linear')

    # Plot original TS
    fig, axs = plt.subplots(1, 3, figsize=(20,5))
    axs[0].plot(sku_df, label='Original')
    axs[0].legend()
    axs[0].set_title('Original Time Series')

    # Copy the original df
    sku_df_log = sku_df.copy()

    # Apply log transformation to the 'Q' column in the new df, replace negative infinity values with NaN and impute 
    sku_df_log['Q'] = np.log1p(sku_df_log['Q'])
    sku_df_log['Q'] = sku_df_log['Q'].replace(-np.inf, np.nan)
    sku_df_log = sku_df_log.interpolate(method='linear')

    # Plot log transformed TS
    axs[1].plot(sku_df_log, label='Log applied')
    axs[1].legend()
    axs[1].set_title('Logarithmic data')


    # Stationarity check using seperate function based on ADF
    def check_stationarity(series):
        result = adfuller(series)
        significance_level = 0.05
        return result[1] < significance_level

    # Initial preprocess level = 0
    preprocess_level = 0

    # Run check_stationarity on log transformed ts
    is_stationary = check_stationarity(sku_df_log['Q'])

    # If data is not stationary, apply first differencing on both trend and seasonality and change preprocess level to 1
    if not is_stationary:
        sku_df_diff = sku_df_log.diff().diff(periods=1)
        sku_df_diff.dropna(inplace=True)
        preprocess_level = 1

        # Run check_stationary on first difference data
        is_stationary = check_stationarity(sku_df_diff['Q'])
        
        # If the data is stationary, continue, if not, apply second differencing on both trend and seasonality and change preprocess level to 2
        if is_stationary:
            print("The time series is now stationary after differencing.")
            sku_df_stationary = sku_df_diff

        else:
            sku_df_diff2 = sku_df_diff.diff().diff(periods=1)
            sku_df_diff2.dropna(inplace=True)
            preprocess_level = 2
            
            # Run check_stationary on second difference data
            is_stationary = check_stationarity(sku_df_diff2['Q'])
            
            # If the data is stationary, continue, if not, print warning and apply second differencing on both trend and seasonality and change preprocess level to 2
            if is_stationary:
                print("The time series is now stationary after second differencing.")
                sku_df_stationary = sku_df_diff2
            else:
                print("The time series is still not stationary after second differencing. Keeping the second differencing data")
                sku_df_stationary = sku_df_diff2
    
    # If the data is stationary from the beginning, do nothing and keep preprocessing level at 0
    else:
        print("The time series is already stationary.")
        sku_df_stationary = sku_df_log

    # Plot after stationarity check and conversion
    axs[2].plot(sku_df_stationary, label='Stationary')
    axs[2].legend()
    axs[2].set_title('Stationary time series')
    plt.show()

    return sku_df_stationary, preprocess_level


def metrics_calc_print(model, test_data, pred_data):
    """
    Calculating performance metrics RMSE, MSE, and MAE

    :parameters:
    - Name of model given as string
    - Test data
    - Predicted data

    :return:
    - Print statement with metrics
    - RMSE, MSE, MAE values
    """
    print('Calculating metrics...')
  
    # Invert log transformation
    test_data = np.expm1(test_data)
    pred_data = np.expm1(pred_data)

    # Impute negative values with 0 (or another chosen value)
    test_data = np.where(test_data < 0, 0, test_data)
    pred_data = np.where(pred_data < 0, 0, pred_data)
    test_data = np.where(np.isinf(test_data), 0, test_data)
    pred_data = np.where(np.isinf(pred_data), 0, pred_data)

    # Calculate metrics
    rmse = math.sqrt(mean_squared_error(test_data, pred_data))
    mse =  mean_squared_error(test_data, pred_data)
    mae = arima_mae = mean_absolute_error(test_data, pred_data)

    print(f'{model} RMSE: {rmse}')
    print(f'{model} MSE: {mse}')
    print(f'{model} MAE: {mae}')

    return rmse, mse, mae


def run_arima(train, test):
    """ 
    Function for training, tuning, and testing ARIMA

    :parameters:
    - Training data
    - Test data

    :returns:
    - Best parameters
    - Prediction data
    - Test data
    """

    print('\n')
    print('************************************')
    print('ARIMA')
    print('************************************')

    np.random.seed(42)
    
    train_arima = train.copy()
    test_arima = test.copy()

    # Fitting arima model, return and extract the best parameters
    arima_model = auto_arima(train_arima, return_best_params=True)
     
    # Extract the best ARIMA parameters
    best_arima_model = (arima_model.order)

    # Generate predictions for the test set
    arima_test_predictions = arima_model.predict(n_periods=len(test_arima))

    return best_arima_model, arima_test_predictions, test_arima


def run_es(train,test):
    """ 
    Function for training, tuning, and testing Exponential smoothing

    :parameters:
    - Training data
    - Test data

    :returns:
    - Best parameters
    - Prediction data
    - Test data
    """
    print('\n')
    print('************************************')
    print('Exponential smoothing')
    print('************************************')

    np.random.seed(42)

    # Define the hyperparameters to tune over
    alpha_range_es = np.linspace(0.1, 1, 5)
    beta_range_es = np.linspace(0.1, 1, 5)
    gamma_range_es = np.linspace(0.1, 1, 5)
    trend_options = ['add', 'mul', None]
    seasonal_options = ['add', 'mul', None]
    damped_trend_options = [True, False]

    # Create hyperparametergrid
    hyperparameters_es = [(alpha, beta, gamma, trend, seasonal, damped_trend) for alpha in alpha_range_es for beta in beta_range_es for gamma in gamma_range_es for trend in trend_options for seasonal in seasonal_options for damped_trend in damped_trend_options]

    train_es = train.copy()
    test_es = test.copy()

    best_rmse = float('inf')

    # Train and test models with different hyperparameter combinations to find the combination with the best RMSE
    for alpha, beta, gamma, trend, seasonal, damped_trend in hyperparameters_es:

        try:
            model = ExponentialSmoothing(train_es, trend=trend, seasonal=seasonal, seasonal_periods=12, damped_trend=damped_trend).fit(smoothing_level=alpha, smoothing_slope=beta, smoothing_seasonal=gamma)
            es_validation_predictions = model.forecast(len(test_es))
            es_rmse = math.sqrt(mean_squared_error(test_es, es_validation_predictions))

            # If the current combination has an RMSE that is lower than the current best_rmse then these hyperparameters should be saved
            if es_rmse < best_rmse:
                best_rmse = es_rmse
                es_best_hyperparameters = (alpha, beta, gamma, trend, seasonal, damped_trend)

        except ValueError:
            continue

    # Fit the model with the best hyperparameters on the training set
    model = ExponentialSmoothing(train_es).fit(smoothing_level=es_best_hyperparameters[0], smoothing_slope=es_best_hyperparameters[1], smoothing_seasonal=es_best_hyperparameters[2])

    # Generate predictions for the test set
    es_test_predictions = model.forecast(len(test_es))

    return es_best_hyperparameters, es_test_predictions, test_es

def run_xgb(train, test, tscv):
      """ 
      Function for training, tuning, and testing XGboost

      :parameters:
      - Training data
      - Test data

      :returns:
      - Best parameters
      - Prediction data
      - Test data
      """
      print('\n')
      print('************************************')
      print('XGBOOST')
      print('************************************')

      # Define hyperparmeter grid
      xgb_params = {
          'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.4],
          'n_estimators': [10, 20, 50, 100, 200],
          'max_depth': [3, 5, 7, 9, 12],
          'colsample_bytree': [0.3, 0.4, 0.5, 0.7, 0.8],
          'min_child_weight': [1, 3, 5, 7, 9],
          'lambda': [0.01, 0.1, 1.0, 10.0, 100.0],
          'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
      }
      
      np.random.seed(42)

      train_xbg=train.copy()
      test_xbg=test.copy()
      
      # Converting index into unix time format as required for XGB
      train_xbg.index=train_xbg.index.astype(np.int64) // 10**9
      test_xbg.index=test_xbg.index.astype(np.int64) // 10**9

      # Splitting into X and y for training and testing sets
      xgb_train_x = train_xbg.index.values.reshape(-1, 1)
      xgb_train_y = train_xbg['Q']
      xgb_test_x = test_xbg.index.values.reshape(-1, 1)
      xgb_test_y = test_xbg['Q']

      # Initiate XGBoost object
      xgboost_model = xgb.XGBRegressor(objective='reg:squarederror')

      # Define the random search object. Input is the model, the parameters to search over, the cross-validation method to apply, and what scoring method that evaluates the performance of each set of hyperparameters
      random_search = RandomizedSearchCV(xgboost_model, param_distributions=xgb_params, cv=tscv, scoring='neg_mean_squared_error', n_iter = 50)

      #Convert the input features and target values to DMatrix objects
      dmat_train = xgb.DMatrix(xgb_train_x, label=xgb_train_y)
      dmat_test = xgb.DMatrix(xgb_test_x, label=xgb_test_y)
      
      # Find the best parameters based on the random_search object
      random_search.fit(xgb_train_x,xgb_train_y)
      best_xgboost_model = random_search.best_estimator_

      #Generate predictions for the test set
      xgboost_test_predictions = best_xgboost_model.predict(xgb_test_x)

      return best_xgboost_model, xgboost_test_predictions, xgb_test_y


def run_prophet(train, test):
    """ 
    Function for training, tuning, and testing Prophet

    :parameters:
    - Training data
    - Test data

    :returns:
    - Best parameters
    - Prediction data
    - Test data
    """
    print('\n')
    print('************************************')
    print('PROPHET')
    print('************************************')

    np.random.seed(42)

    # Define hyperparmeter grid
    proph_params = {
        'seasonality_mode': ['additive', 'multiplicative'],
        'changepoint_prior_scale': [0.001, 0.1],
        'seasonality_prior_scale': [0.01, 1.0],
        'changepoint_range': [0.8, 0.9],
        'growth': ['linear', 'logistic']
    }

    train_proph = train.copy()
    test_proph = test.copy()

    # Renaming to required Prophet column titles
    train_proph = train_proph.reset_index().rename(columns={"date_": "ds","Q": "y"})
    test_proph = test_proph.reset_index().rename(columns={"date_": "ds","Q": "y"})

    best_proph_params = None
    best_prophet_rmse = None

    # Loop over hyperparameter grid
    for mode in proph_params['seasonality_mode']:
        for prior_scale in proph_params['changepoint_prior_scale']:
            for seasonal_scale in proph_params['seasonality_prior_scale']:
                for changepoint in proph_params['changepoint_range']:
                    for growth in proph_params['growth']:

                        # Fit Prophet model with the selected hyperparameters
                        prophet_model = Prophet(seasonality_mode=mode,
                                                changepoint_prior_scale=prior_scale,
                                                seasonality_prior_scale=seasonal_scale,
                                                changepoint_range=changepoint,
                                                growth=growth,
                                                yearly_seasonality=True)
                        prophet_model.fit(train_proph)

                        # Generate predictions for the test set and calculate RMSE
                        future = prophet_model.make_future_dataframe(periods=len(test_proph), freq='MS')
                        future = future[-len(test_proph):] 
                        forecast = prophet_model.predict(future)
                        prophet_rmse = math.sqrt(mean_squared_error(test_proph.y, forecast.yhat))

                        # If the RMSE is lower than current best RMSE, overwrite the parameters
                        if best_proph_params is None:
                            best_proph_params = {'rmse': prophet_rmse, 
                                                    'seasonality_mode': mode, 
                                                    'changepoint_prior_scale': prior_scale, 
                                                    'seasonality_prior_scale': seasonal_scale, 
                                                    'changepoint_range': changepoint,
                                                    'growth': growth}
                        elif prophet_rmse < best_proph_params['rmse']:
                            best_proph_params = {'rmse': prophet_rmse, 
                                                    'seasonality_mode': mode, 
                                                    'changepoint_prior_scale': prior_scale, 
                                                    'seasonality_prior_scale': seasonal_scale, 
                                                    'changepoint_range': changepoint,
                                                    'growth': growth}
                        elif prophet_rmse > best_proph_params['rmse']:
                            pass

    return best_proph_params, forecast, test_proph


def run_lstm(train, test, sku):
      """ 
      Function for training, tuning, and testing Prophet

      :parameters:
      - Training data
      - Test data

      :returns:
      - Best parameters
      - Prediction data
      - Test data
      """
      print('\n')
      print('************************************')
      print('LSTM')
      print('************************************')

      np.random.seed(42)

      train_lstm = train.copy()
      test_lstm = test.copy()

      # Try except block to handle SKUs with fewer historical datapoints
      try:
          # Preprocessing input data into seq 15 using support function
          train_lstm_x,train_lstm_y,scaler_train = lstm_prepros_seq(train_lstm, 15)
          test_lstm_x,test_lstm_y,scaler_test = lstm_prepros_seq(test_lstm, 15)

          # LSTM requires a 3D shape input (samples, features, timesteps) 
          # samples = number of observations, features = 1 (univariate analysis), timesteps = number of timesteps in each sequence so equal to seq parameter
          train_lstm_x = np.reshape(train_lstm_x, (train_lstm_x.shape[0], train_lstm_x.shape[1], 1))
          test_lstm_x = np.reshape(test_lstm_x, (test_lstm_x.shape[0], test_lstm_x.shape[1], 1))
          
          # Hyperparametertuning using model_tuning function
          objective = Objective('mse', direction='min')

          # using timestamp as unique_id for given model
          unique_id = int(time.time())
          tuner = kt.RandomSearch(lambda hp: model_tuning(hp, train_lstm_x, train_lstm_y), objective=objective, max_trials = 3, executions_per_trial = 1, directory=f"./{unique_id}")

          # Define the checkpoint callback to save the best model
          checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(f"{sku}_{unique_id}_lstm_best_model.h5", monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

          # Set up the checkpoint directory
          checkpoint_dir = os.path.dirname(f"{sku}_{unique_id}_lstm_best_model.h5")

          # Define the EarlyStopping callback
          early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

          # Train the model with the checkpoint callback
          tuner.search(x=train_lstm_x, y=train_lstm_y, epochs=15, batch_size=64, validation_data=(test_lstm_x, test_lstm_y), callbacks=[checkpoint_callback, early_stopping_callback])

          # Extracting the best model
          saved_best_model_path = f"{sku}_{unique_id}_lstm_best_model.h5"
          lstm_best_model = lstm_best_model = load_model(saved_best_model_path)

          # Using the best model to predict using the test set
          test_pred = lstm_best_model.predict(test_lstm_x)

          # Undo scaling of variables before calculating metrics
          test_y = scaler_test.inverse_transform(test_lstm_y.reshape(-1, 1))
          test_pred = scaler_test.inverse_transform(test_pred)

      # If not enough data for seq = 15, trying with a smaller window (seq = 5)
      except IndexError:
        try:
            train_lstm_x, train_lstm_y, scaler_train = lstm_prepros_seq(train_lstm, 5)
            test_lstm_x, test_lstm_y, scaler_test = lstm_prepros_seq(test_lstm, 5)

            train_lstm_x = np.reshape(train_lstm_x, (train_lstm_x.shape[0], train_lstm_x.shape[1], 1))
            test_lstm_x = np.reshape(test_lstm_x, (test_lstm_x.shape[0], test_lstm_x.shape[1], 1))

            # Hyperparametertuning using model_tuning function
            objective = Objective('mse', direction='min')

            # using timestamp as unique_id for given model
            unique_id = int(time.time())
            tuner = kt.RandomSearch(lambda hp: model_tuning(hp, train_lstm_x, train_lstm_y), objective=objective, max_trials = 3, executions_per_trial = 1, directory=f"./{unique_id}")

            # Define the checkpoint callback to save the best model
            checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(f"{sku}_{unique_id}_lstm_best_model.h5", monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

            # Set up the checkpoint directory
            checkpoint_dir = os.path.dirname(f"{sku}_{unique_id}_lstm_best_model.h5")

            # Train the model with the checkpoint callback
            tuner.search(x=train_lstm_x, y=train_lstm_y, epochs=15, batch_size=64, validation_data=(test_lstm_x, test_lstm_y), callbacks=[checkpoint_callback])

            # Extracting the best model
            saved_best_model_path = f"{sku}_{unique_id}_lstm_best_model.h5"
            lstm_best_model = lstm_best_model = load_model(saved_best_model_path)

            # Using the best model to predict using the test set
            test_pred = lstm_best_model.predict(test_lstm_x)

            # Undo scaling of variables before calculating metrics
            test_y = scaler_test.inverse_transform(test_lstm_y.reshape(-1, 1))
            test_pred = scaler_test.inverse_transform(test_pred)

        # if seq = 5 not feasible either, the model will be skipped and metrics set at -1
        except IndexError:
            lstm_rmse, lstm_mse, lstm_mae = -1, -1, -1
            lstm_best_model = None
            test_pred = None
            test_y = None

      return lstm_best_model, test_pred, test_y

def invert_preprocessing(y_pred, preprocess_level, periods_to_forecast, sku_df):
    """
    Function to invert preprocessing, specifically the differencing conducted to make the series stationary
    Applied after forecasting

    :parameters:
    - Predicted values
    - Preprocessing level 
    - Number of periods forecasted
    - SKU level dataframe

    :return:
    - Inverted predictions
    """

    y_pred = np.expm1(y_pred)

    # If first difference was conducted
    if preprocess_level == 1:
      y_pred = np.cumsum(y_pred)[-periods_to_forecast:]
      y_pred = np.insert(y_pred, 0, sku_df['Q'].iloc[-1])
      y_pred = y_pred[1:]

    # If second difference was conducted
    elif preprocess_level == 2:
      y_pred = np.cumsum(y_pred)[-periods_to_forecast:]
      y_pred = np.insert(y_pred, 0, sku_df['Q'].iloc[-1])
      y_pred = np.cumsum(y_pred)
      y_pred = np.insert(y_pred, 0, sku_df['Q'].iloc[-2])
      y_pred = y_pred[-periods_to_forecast:]
      y_pred = y_pred

    # If no difference was conducted
    else:
      y_pred = y_pred[-periods_to_forecast:]
      
    return y_pred


## Main function for training and testing

In [ ]:
def train_test_models(df):
  """
  Main function for training, tuning and testing ARIMA, ETS, XGB, Prophet and LSTM on SKU level
  The main function applies the following support functions: preprocess(), lstm_prepros_seq(), model_tuning(), run_arima(), run_es(), run_xgb(), run_prophet(), run_lstm()

  :parameters:
  - DataFrame containing historical sales for all SKUs

  :return:
  - Dictionary containing the performance of all 5 models for all SKUs
  """

  np.random.seed(42)

  # Include only relevant columns
  df = df[['_ItemNumber','Q','date_']]

  # Initate dictionary to store performance metrics
  output_dict = {}

  # Looping through each SKU
  for sku in df['_ItemNumber'].unique():

    # Initiating outer dict to store specific SKU information
    output_dict[sku]={'metrics': {}, 'parameters': {}}

    print('************************************')
    print(f'Processing SKU:{sku}')
    print('************************************')
    print('\n')

    # Preprocess SKU df and save the level of preprocessing for later inverting
    sku_df, preprocess_level = preprocess(df, sku)

    # Cross validation using TimeSeriesSplit to make sure we keep the chronological order. For each SKU it will create 3 splits of the data into training and testing sets
    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)  
    
    # List comprehension for storing RMSE, MSE and MAE for each fold
    arima_all_rmse, es_all_rmse, xgboost_all_rmse, prophet_all_rmse, lstm_all_rmse, arima_all_mse, es_all_mse, xgboost_all_mse, prophet_all_mse, lstm_all_mse, arima_all_mae, es_all_mae, xgboost_all_mae, prophet_all_mae, lstm_all_mae = [[] for _ in range(15)]  


    ######### Training and testing #########
    # For each model: train, tune, test, calculate metrics, and append to metric lists
    for fold, (train_index, test_index) in enumerate(tscv.split(sku_df)):
      
      num_folds = tscv.get_n_splits()
      
      print(f'FOLD {fold + 1} / {num_folds} ')

      # Split the data into train and test sets for the current fold
      train = sku_df.iloc[train_index]
      test = sku_df.iloc[test_index]
      
      ######### ARIMA #########
      
      best_arima_model, arima_test_predictions, test_arima = run_arima(train, test)
  
      arima_rmse, arima_mse, arima_mae = metrics_calc_print('ARIMA', test_arima, arima_test_predictions)

      arima_all_rmse.append(arima_rmse)
      arima_all_mse.append(arima_mse)
      arima_all_mae.append(arima_mae)


      ######### EXP SMOOTHING #########

      es_best_hyperparameters, es_test_predictions, test_es = run_es(train, test)

      es_rmse, es_mse, es_mae = metrics_calc_print('EXPONENTIAL SMOOTHING', test_es, es_test_predictions)

      es_all_rmse.append(es_rmse)
      es_all_mse.append(es_mse)
      es_all_mae.append(es_mae)


      ######### XGBOOST #########

      try:
        best_xgboost_model, xgboost_test_predictions, xgb_test_y = run_xgb(train, test, tscv)

        xgboost_rmse, xgboost_mse, xgboost_mae = metrics_calc_print('XGBOOST', xgb_test_y, xgboost_test_predictions)

        xgboost_all_rmse.append(xgboost_rmse)
        xgboost_all_mse.append(xgboost_mse)
        xgboost_all_mae.append(xgboost_mae)

      except ValueError as ve:
          print(f"Skipping XGBoost for SKU {sku}: {ve}")
          continue


      ######### Prophet #########

      prophet_best_model, forecast, test_proph = run_prophet(train, test)

      prophet_rmse, prophet_mse, prophet_mae = metrics_calc_print('PROPHET', test_proph.y, forecast.yhat)

      prophet_all_rmse.append(prophet_rmse)
      prophet_all_mse.append(prophet_mse)
      prophet_all_mae.append(prophet_mae)
      
      ###### LSTM ######

      lstm_best_model, test_predictions, test_y = run_lstm(train, test, sku)

      if test_y is None:
          lstm_rmse, lstm_mse, lstm_mae = -1, -1, -1
          print('Warning: no metrics calculated. Too little data to process sequentially')

      else:
          lstm_rmse, lstm_mse, lstm_mae = metrics_calc_print('LSTM', test_y, test_predictions)

      lstm_all_rmse.append(lstm_rmse)
      lstm_all_mse.append(lstm_mse)
      lstm_all_mae.append(lstm_mae)

    print('\n')
    print('Finalizing results...')
    print('\n')
    print(f'Average scores across {num_folds} folds')
    print('************************************')
    
    ######### Saving results #########

    # Define the list of models and their corresponding metric lists
    models = ['arima', 'es', 'xgboost', 'prophet', 'lstm']
    all_metrics = {'rmse': [arima_all_rmse, es_all_rmse, xgboost_all_rmse, prophet_all_rmse, lstm_all_rmse],
                    'mse': [arima_all_mse, es_all_mse, xgboost_all_mse, prophet_all_mse, lstm_all_mse],
                    'mae': [arima_all_mae, es_all_mae, xgboost_all_mae, prophet_all_mae, lstm_all_mae]}

    # Calculate the average performance of the models across all folds
    averaged_metrics = {model: {metric: np.nanmean(all_metrics[metric][i]) for metric in all_metrics} for i, model in enumerate(models)}

    # Saving metrics for each model
    for model in models:
      output_dict[sku]['metrics'][model] = averaged_metrics[model]

    # Saving best parameters for each model in the output dict
    output_dict[sku]['parameters']['arima'] = best_arima_model
    output_dict[sku]['parameters']['es'] = es_best_hyperparameters
    output_dict[sku]['parameters']['xgboost'] = best_xgboost_model.get_params()
    output_dict[sku]['parameters']['prophet'] = prophet_best_model
    if lstm_best_model:
        output_dict[sku]['parameters']['lstm'] = lstm_best_model.get_config()
    else:
        output_dict[sku]['parameters']['lstm'] = None


    print('\n')
    print('Updated dictionary...')
    
  return output_dict



## Main function for forecasting

In [ ]:
def deploy_forecast(model_dict, df, periods_to_forecast):
    """
    Main function for deploying forecast based on the output from the training and testing function
    The best performing model based on RMSE is applied for the final forecast
    
    :parameters:
    - Output dictionary from train_test function with performance metrics on SKU level
    - DataFrame with SKU level sales data
    - Number of periods to forecast

    :returns:
    - 


    """
    np.random.seed(42) 

    forecast_df = pd.DataFrame(columns=['_ItemNumber', 'Model', 'Forecast', 'ConfInt_lower', 'ConfInt_upper', 'train_test_RMSE'])

    unique_skus = df['_ItemNumber'].unique()

    ######### Deploy forecast #########
    for sku in unique_skus:

        # Subsetting SKU level data
        sku_df = df[df['_ItemNumber'] == sku]

        # Preprocess data
        sku_df, preprocess_level = preprocess(sku_df, sku)

        # Extract information about best_model and best_model metrics
        metrics = model_dict[sku]['metrics']

        # Find then best model as the one with the minimum RMSE and extract its respective best parameters
        best_model = min([model for model in metrics if metrics[model]['rmse'] >= 0], key=lambda x: metrics[x]['rmse'])
        best_params = model_dict[sku]['parameters'][best_model]
        best_performance = model_dict[sku]['metrics'][best_model]['rmse']
        
        # Deploy forecast based on best model
        if best_model == 'arima':
            print(f'Forecasting sku {sku} using ARIMA')

            # Extracting best parameters
            p, d, q = best_params
            model = ARIMA(sku_df, order=(p, d, q))
            results = model.fit()
            
            # Forecast values + confidence intervals
            forecast_obj = results.get_forecast(steps=periods_to_forecast, alpha=0.05)
            forecast = forecast_obj.predicted_mean 
            conf_int = forecast_obj.conf_int()

            # Invert preprocessing 
            forecast = forecast.values
            forecast = invert_preprocessing(forecast, preprocess_level, periods_to_forecast, sku_df)
            conf_int_lower = invert_preprocessing(conf_int.iloc[:,0].values, preprocess_level, periods_to_forecast, sku_df)
            conf_int_upper = invert_preprocessing(conf_int.iloc[:,1].values, preprocess_level, periods_to_forecast, sku_df)
            conf_int = pd.DataFrame({'ConfInt_lower':conf_int_lower , 'ConfInt_upper':conf_int_upper, 'train_test_RMSE':best_performance})

            # Creating a DataFrame to store the forecast results incl confidenct intervalt
            forecast_index = pd.date_range(start=sku_df.index[-1] + pd.Timedelta(days=7), periods=periods_to_forecast + 1, closed='right', freq='W')
            sku_forecast = pd.DataFrame({'_ItemNumber': sku, 'Model': 'ARIMA', 'Forecast': forecast})
            sku_forecast = sku_forecast.join(conf_int)
            sku_forecast = sku_forecast.set_index(forecast_index)

            # Renaming the conf_int columns to align with main dataframe
            sku_forecast = sku_forecast.rename(columns={'lower Q': 'ConfInt_lower', 'upper Q': 'ConfInt_upper'})
            
            # Appending forecast results to the forecast_df
            forecast_df = pd.concat([forecast_df, sku_forecast], axis=0)

            print(sku)
            print(sku_forecast)
            

        elif best_model == 'es':
            print(f'Forecasting sku {sku} using Exponential Smoothing')
            
            sku_df = sku_df.clip(lower=0.01)
            
            # Extract best parameters
            alpha, beta, gamma, trend, seasonal, damped_trend = best_params

            # First trying to fit the strand and the seasonal component
            try:
                model = ExponentialSmoothing(sku_df, trend=trend, seasonal=seasonal, damped_trend=damped_trend)
                results = model.fit(smoothing_level=alpha, smoothing_slope=beta, smoothing_seasonal=gamma)

            # If there is not sufficient data to fit the seasonal component, thus the specific error gets thrown, the seasonal component gets removes
            except ValueError as e:
                if 'Cannot compute initial seasonals using heuristic method' in str(e):
                    
                    model = ExponentialSmoothing(sku_df, trend=trend, seasonal=None, damped_trend=damped_trend)

                    results = model.fit(smoothing_level=alpha, smoothing_slope=beta)
                
                # If the valueerror does not equal the above, then the alternative error should get thrown
                else:
                    raise e
             
            # Create forecasts      
            forecast = results.forecast(periods_to_forecast)

            # Invert preprocessing
            forecast = forecast.values
            forecast = invert_preprocessing(forecast, preprocess_level, periods_to_forecast, sku_df)

            # Calculate the confidence intervals manually
            stderr = np.std(results.resid, ddof=1)  
            conf_int = np.outer(forecast, np.array([1, 1])) + stderr * np.array([-1.96, 1.96])

            # Create dataframe to store forecasts
            forecast_index = pd.date_range(start=sku_df.index[-1] + pd.Timedelta(days=7), periods=periods_to_forecast + 1, closed='right', freq='W')
            sku_forecast = pd.DataFrame({'_ItemNumber': sku, 'Model': 'EXP SMOOTH', 'Forecast': forecast, 'ConfInt_lower': conf_int[:, 0], 'ConfInt_upper': conf_int[:, 1], 'train_test_RMSE':best_performance})
            sku_forecast = sku_forecast.set_index(forecast_index)

            # Appending forecast results to the forecast_df
            forecast_df = pd.concat([forecast_df, sku_forecast], axis=0)

            print(sku)
            print(sku_forecast)
            
        
        elif best_model == 'xgboost':
            print(f'Forecasting sku {sku} using XGBOOST')

            # Reset the index and convert the index to a new column 'Date'
            sku_df.reset_index(inplace=True)
            sku_df.rename(columns={'index': 'date_'}, inplace=True)
            
            # Convert date_ to unix timestamp because xgb only takes numerival data
            sku_df['date_'] = sku_df['date_'].astype(np.int64) // 10**9

            # Create the X and y matrices for the model
            X = sku_df['date_']
            y = sku_df['Q']

            # Create XGBoost model with best parameters
            booster = xgb.train(params=best_params, dtrain=xgb.DMatrix(X, label=y))

            # Creating the forecast horizon date range for the forecasts
            start_date = pd.to_datetime(sku_df['date_'], unit='s').max() + pd.Timedelta(weeks=1)
            end_date = start_date + pd.Timedelta(weeks=periods_to_forecast)
            forecast_dates = pd.date_range(start=start_date, end=end_date, freq='W')

            # Create the forecast_dates_df DataFrame with the dates as the 'date_' column
            forecast_dates_df = pd.DataFrame({'date_': forecast_dates})
            forecast_dates_df['date_'] = forecast_dates_df['date_'].astype(int)

            # Convert the forecast_dates_df DataFrame to a DMatrix object
            forecast_dmatrix = xgb.DMatrix(data=forecast_dates_df.values)

            # Make predictions using the pre-trained XGBoost model
            forecast = booster.predict(forecast_dmatrix, ntree_limit=booster.best_ntree_limit)
            forecast_stdevs = booster.predict(forecast_dmatrix, ntree_limit=booster.best_ntree_limit, pred_contribs=True).std(axis=1)[:periods_to_forecast]

            # Invert preprocessing
            forecast = invert_preprocessing(forecast, preprocess_level, periods_to_forecast, sku_df)

            # Calculate upper and lower bounds of 95% confidence interval
            upper_bound = forecast + 1.96 * forecast_stdevs
            lower_bound = forecast - 1.96 * forecast_stdevs

            # Combine forecast and confidence intervals into DataFrame
            sku_df.index = pd.to_datetime(sku_df['date_'], unit='s')
            forecast_index = pd.date_range(start=sku_df.index[-1] + pd.Timedelta(days=7), periods=periods_to_forecast + 1, closed='right', freq='W')
            sku_forecast = pd.DataFrame({'_ItemNumber': sku, 'Model': 'XGBOOST', 'Forecast': forecast, 'ConfInt_lower': upper_bound, 'ConfInt_upper': lower_bound,'train_test_RMSE':best_performance })
            sku_forecast = sku_forecast.set_index(forecast_index)

            # Appending forecast results to the forecast_df
            forecast_df = pd.concat([forecast_df, sku_forecast], axis=0)

            print(sku)
            print(sku_forecast)
            
        
        elif best_model == 'prophet':
            print(f'Forecasting sku {sku} using prophet')

            # Change naming convention to fit to proplet
            sku_df_prophet = pd.DataFrame({'ds': sku_df.index, 'y': sku_df.values.flatten()})

            # Train model with best parameters
            model = Prophet(changepoint_prior_scale=best_params['changepoint_prior_scale'], seasonality_prior_scale=best_params['seasonality_prior_scale'], changepoint_range=best_params['changepoint_range'], seasonality_mode=best_params['seasonality_mode'], interval_width=0.95)
            model.fit(sku_df_prophet)

            # Make weekly predictions
            future = model.make_future_dataframe(periods=periods_to_forecast, freq='W',include_history=False)
            forecast = model.predict(future)

            # Invert preprocessing
            forecast['yhat'] = invert_preprocessing(forecast['yhat'], preprocess_level, periods_to_forecast, sku_df)

            # Calculate the confidence intervals
            conf_int = forecast[['yhat_lower', 'yhat_upper']].values
            conf_int = invert_preprocessing(conf_int, preprocess_level, periods_to_forecast, sku_df)
            forecast['yhat_lower'] = conf_int[:, 0]
            forecast['yhat_upper'] = conf_int[:, 1]
            
            # Create forecast df
            sku_forecast = pd.DataFrame({'date': forecast['ds'], '_ItemNumber': sku, 'Model': 'PROPHET', 'Forecast': forecast['yhat'], 'ConfInt_lower': forecast['yhat_lower'], 'ConfInt_upper': forecast['yhat_upper'],'train_test_RMSE':best_performance })
            sku_forecast = sku_forecast.set_index('date')

            # Appending forecast results to the forecast_df
            forecast_df = pd.concat([forecast_df, sku_forecast], axis=0)
            print(sku)
            print(sku_forecast)


        elif best_model == 'lstm':
            print(f'Forecasting sku {sku} using LSTM')


            np.random.seed(42)
            tf.random.set_seed(42)
            
            # Create range of dates used for the forecasting df later
            forecast_index = pd.date_range(start=sku_df.index[-1], periods=periods_to_forecast + 1, closed='right', freq='W')

            # Retrieving best parameters and configuring the model
            model_params = model_dict[sku]['parameters'][best_model]
            model = keras.models.Sequential.from_config(model_params)
            model.compile(loss='mse', optimizer='adam')

            # Determine the sequence length based on the trained LSTM model's input shape
            seq_length = model.layers[0].input_shape[1]

            # Preprocessing input data into sequential
            X, y, scaler = lstm_prepros_seq(sku_df, seq=seq_length)

            # Reshape X
            X = np.reshape(X, (X.shape[0], X.shape[1], 1))

            # Fit model
            model.fit(X, y, epochs=20, verbose=0)

            # Make out-of-sample forecasts
            X_forecast = sku_df['Q'][-seq_length:].values.reshape(1, seq_length, 1)
            y_pred = []

            for i in range(periods_to_forecast):
                forecast = model.predict(X_forecast)
                y_pred.append(forecast[0, 0])
                X_forecast = np.roll(X_forecast, -1)
                X_forecast[0, -1, 0] = forecast[0, 0]


            y_pred = np.array(y_pred)

            # Invert scaling
            y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

            # Invert preprocessing
            y_pred = invert_preprocessing(y_pred ,preprocess_level, periods_to_forecast, sku_df)
            
            # Calculate residuals 
            residuals = y[-periods_to_forecast:] - y_pred.flatten()
            se = np.std(residuals)

            # Calculate the confidence intervals
            alpha = 0.05
            z = norm.ppf(1 - alpha / 2)
            lower_bounds = y_pred - z * se
            upper_bounds = y_pred + z * se

            sku_forecast = pd.DataFrame({'_ItemNumber':sku, 'Model': 'LSTM', 'Forecast': y_pred.flatten(),'ConfInt_lower': lower_bounds.flatten(),'ConfInt_upper': upper_bounds.flatten(),'train_test_RMSE':best_performance }, index=forecast_index)

            forecast_df = pd.concat([forecast_df, sku_forecast], axis=0)
            
            print(sku)
            print(sku_forecast)
        
    return forecast_df